## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-01-22-52-18 +0000,nypost,NYPD detective removed from Israel post but wi...,https://nypost.com/2025/11/01/us-news/israel-l...
1,2025-11-01-22-39-22 +0000,nyt,Explosion at Harvard Medical School Was ‘Inten...,https://www.nytimes.com/2025/11/01/us/harvard-...
2,2025-11-01-22-29-28 +0000,nypost,Left-wing podcaster sparks outrage with meme d...,https://nypost.com/2025/11/01/us-news/left-win...
3,2025-11-01-22-25-50 +0000,nypost,Ukrainians turn to Tarot readings — to find ou...,https://nypost.com/2025/11/01/world-news/ukrai...
4,2025-11-01-22-02-04 +0000,nypost,Trump threatens to go into Nigeria ‘guns-a-bla...,https://nypost.com/2025/11/01/us-news/trump-th...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,25
132,new,14
147,election,10
145,he,10
106,snap,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
197,2025-10-31-23-04-00 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...,66
21,2025-11-01-20-44-56 +0000,bbc,US judges say Trump administration must contin...,https://www.bbc.com/news/articles/cr433x9zqq4o...,61
9,2025-11-01-21-48-00 +0000,nypost,"Trump to host Ahmad al-Sharaa, marking first-e...",https://nypost.com/2025/11/01/us-news/trump-to...,59
20,2025-11-01-20-45-56 +0000,nypost,"Great white sharks swarmed New York, New Jerse...",https://nypost.com/2025/11/01/us-news/great-wh...,57
134,2025-11-01-08-43-08 +0000,nyt,Carney Says He Apologized to Trump Over Ad Rev...,https://www.nytimes.com/2025/11/01/world/canad...,55


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
197,66,2025-10-31-23-04-00 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...
20,57,2025-11-01-20-45-56 +0000,nypost,"Great white sharks swarmed New York, New Jerse...",https://nypost.com/2025/11/01/us-news/great-wh...
15,50,2025-11-01-20-55-31 +0000,nypost,House Republicans exploring ways to prevent Ma...,https://nypost.com/2025/11/01/us-news/house-re...
177,31,2025-11-01-00-09-02 +0000,nypost,JD Vance dresses up like meme-version of himse...,https://nypost.com/2025/10/31/us-news/jd-vance...
164,31,2025-11-01-02-10-13 +0000,bbc,"'We need food, we have no food' - desperation ...",https://www.bbc.com/news/articles/c0jdd186l0go...
6,26,2025-11-01-21-58-22 +0000,nypost,George Santos tells Tucker Carlson prison was ...,https://nypost.com/2025/11/01/us-news/george-s...
50,26,2025-11-01-18-25-22 +0000,nyt,Two More Suspects Charged in Louvre Heist Case,https://www.nytimes.com/2025/11/01/world/europ...
32,23,2025-11-01-20-13-03 +0000,bbc,"Key town faces 'multi-thousand' Russian force,...",https://www.bbc.com/news/articles/crklljyx6mzo...
190,23,2025-10-31-23-24-00 +0000,wsj,The White House is imposing new restrictions o...,https://www.wsj.com/politics/policy/white-hous...
184,22,2025-10-31-23-39-00 +0000,wsj,Lawmakers are facing urgency to resolve the go...,https://www.wsj.com/politics/policy/trumps-dem...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
